# Purpose: 
1. Process Google Form Responses from pool play
2. output record info (wins/losses, win ratio/loss ratio, points) for display
3. after pool play is over, output record info for seeding next pool play

### To Do:
1. Modify for day 2
2. input from day 2 round robin
3. output to playoff brackets

### Setup output, get team names

In [103]:
from ReadWriteGDocs import write_gdoc, read_gdoc_range, write_gdoc_cell

num_teams = 49 ### 49 for men, 45 for women ###
master_sheet = "Men's Master" ### Change for men vs women ###

num_gold = 8
num_silver = 6
num_bronze = 6

import pandas as pd
import numpy as np

input_ws = "NY Mini 2019 Score Input - Men's Day 2 (Responses)" ### Change for men vs women ###
output_ws = ["NY Mini 2019 - 2nd Round", "NY Mini 2019 - Day 2 Playoffs"]

c=read_gdoc_range(input_ws, "Form Responses 2", 1,2,1,12)
c=c[0]

# get list of team names from Form Values
results = pd.DataFrame(read_gdoc_range("Form Values Add-on - DO NOT DELETE", "Sheet1", 2, 4 ### 4 for men, 5 for women ###
                                       , 1+num_teams, 4 ### 4 for men, 5 for women ###
                                      ), columns=["Team Name"])

results["Court Number"] = ""
results["Wins"] = 0
results["Losses"] = 0
results["Points"] = 0
results["Pool Ranking"] = 0

# get court numbers from master day 2 - gold
courts = pd.DataFrame(read_gdoc_range(output_ws[0], master_sheet, 4, 2, 9, 10))

# iterate each row
for i in range(1,courts.shape[0]):
    for j in range(0, courts.shape[1]):
        team_name = courts.at[i, j]
        
        if (team_name != "X"):
            team_row = results.loc[results["Team Name"] == team_name]
            results.at[team_row.index, "Court Number"] = courts.at[0, j]
            
# get court numbers from master day 2 - silver
courts = pd.DataFrame(read_gdoc_range(output_ws[0], master_sheet, 4, 8, 9, 10))

# iterate each row
for i in range(1,courts.shape[0]):
    for j in range(0, courts.shape[1]):
        team_name = courts.at[i, j]
        
        if (team_name != "X"):
            team_row = results.loc[results["Team Name"] == team_name]
            results.at[team_row.index, "Court Number"] = courts.at[0, j]
            
            
# get court numbers from master day 2 - bronze

## SKIP FOR MENS BRONZE
# courts = pd.DataFrame(read_gdoc_range(output_ws[0], master_sheet, 4, 13, 9, 14))

# # iterate each row
# for i in range(1,courts.shape[0]):
#     for j in range(0, courts.shape[1]):
#         team_name = courts.at[i, j]
        
#         if (team_name != "X"):
#             team_row = results.loc[results["Team Name"] == team_name]
#             results.at[team_row.index, "Court Number"] = courts.at[0, j]

# Drop rows without court number because they are in bronze
results = results[results["Court Number"] != ""]
results = results.reset_index(drop=True)

        
# start processing from row 2
# keep track of rows processed using processed

processed = 0
results

,Team Name,Court Number,Wins,Losses,Points,Pool Ranking
0,Boston Hurricanes Black,CT01,0,0,0,0
1,Boston Knights A,CT04,0,0,0,0
2,Boston Knights B,CT14,0,0,0,0
3,Boston Rising Tide,CT04,0,0,0,0
4,Chicago United,CT02,0,0,0,0
5,DC CYC A,CT02,0,0,0,0
6,DC CYC B,CT13,0,0,0,0
7,DC Jin Long,CT14,0,0,0,0
8,DC MVP A,CT04,0,0,0,0
9,DC MVP B,CT12,0,0,0,0


### Import new Google Form responses

In [104]:
responses_in = pd.DataFrame(read_gdoc_range(input_ws, "Form Responses 2", processed + 2,2, 200, 12), 
                    columns=c)

responses_in.replace('', np.nan, inplace=True)
responses_in.dropna(inplace=True, how='all')

from math import isnan

print ("Need to process " + str(len(responses_in)) + " games")

Need to process 70 games


### Process each Google Form response - update wins and points for teams

In [105]:
responses_trimmed = responses_in.drop(c[2:5], axis=1)
## Convert to numeric
for i in responses_trimmed.columns:
    responses_trimmed[i] = pd.to_numeric(responses_trimmed[i], downcast='integer', errors='ignore')
    
# iterate through each row of responses_trimmed and see if team name is found

for index,row in responses_trimmed.iterrows():
    team_1_name = row["Team 1 Name"]
    team_2_name = row["Team 2 Name"]
    win1 = 0
    loss1 = 0
    win2 = 0
    loss2 = 0
    points1 = 0
    points2 = 0
    
    # process first set
    score_diff = row["Team 1 Score"] - row["Team 2 Score"]
    points1 += score_diff
    points2 -= score_diff
        
    if (score_diff > 0):
        win1 += 1
        loss2 += 1
    elif (score_diff < 0):
        win2 += 1
        loss1 += 1
    ## if score_diff == 0 then do nothing
        
#     # process second set
#     score_diff = row["Team 1 Score 2"] - row["Team 2 Score 2"]
#     points1 += score_diff
#     points2 -= score_diff
    
#     if (score_diff > 0): # team 1 won
#         win1 += 1
#     elif (score_diff < 0): # team 2 won
#         win2 += 1
#     ## if score_diff == 0 then do nothing
                    
    team_1_row = results.loc[results["Team Name"] == team_1_name]
    results.at[team_1_row.index, "Wins"] = team_1_row["Wins"] + win1
    results.at[team_1_row.index, "Points"] = team_1_row["Points"] + points1
    results.at[team_1_row.index, "Losses"] = team_1_row["Losses"] + loss1
    
    team_2_row = results.loc[results["Team Name"] == team_2_name]
    results.at[team_2_row.index, "Wins"] = team_2_row["Wins"] + win2
    results.at[team_2_row.index, "Points"] = team_2_row["Points"] + points2
    results.at[team_2_row.index, "Losses"] = team_2_row["Losses"] + loss2

    
# add to results: wins and points

    processed = processed + 1
        
print("Number of records processed: " + str(len(responses_trimmed)))

print("Total Processed: " + str(processed))

Number of records processed: 70
Total Processed: 70


### Check results and resolve ties between 2nd and 3rd place (determines gold)

In [106]:
#results.sort_values(by=["Pool Ranking", "Wins", "Points"], ascending=[1,0,0])
results.sort_values(by=["Court Number", "Wins", "Points"], ascending=[1,0,0])

# HEAD TO HEAD

# CT01 - three way tie - Boston Hurricanes Black vs Thunder A vs NJ Ronin
# CT02 - legacy vs dc cyc a
# CT02 - flying tigers vs chicago united
# CT03 - strangers blue, ngun lam white, gong hoy
# CT04 - DC MVP A VS Boston Knights A
# CT04 - NY Strangers Black VS Boston Rising Tide

# CT12 - Toronto Ngun Lam Blue VS Toronto United
# CT13 - lochai, connex c, toronto phoenix
# CT14 - NY Vikings SPA VS Boston Knights B VS - DC Yee Fung Toy Thunder B

,Team Name,Court Number,Wins,Losses,Points,Pool Ranking
25,Toronto Connex A,CT01,4,0,49,0
22,Philly CIA A,CT01,3,1,-10,0
0,Boston Hurricanes Black,CT01,1,3,-4,0
10,DC Yee Fung Toy Thunder A,CT01,1,3,-16,0
13,NJ Ronin,CT01,1,3,-19,0
29,Toronto LEGACY,CT02,3,1,24,0
5,DC CYC A,CT02,3,1,18,0
12,Montreal Freemason,CT02,2,2,-4,0
28,Toronto Flying Tigers,CT02,1,3,-10,0
4,Chicago United,CT02,1,3,-28,0


### Sort results by court number, team name

### Update display for standings

In [75]:
# sort by court number and team name

disp_results = results.sort_values(by=["Court Number", "Team Name"]).reset_index(drop=True)

# update display output - court numbers and team names - one time only
# for i in range(0, len(disp_results)):
#     write_gdoc_cell(disp_results.at[i, "Court Number"], output_ws[0], "ScoreDisp", 5+i, 2)  ## 2 for men, 9 for women
#     write_gdoc_cell(disp_results.at[i, "Team Name"], output_ws[0], "ScoreDisp", 5+i, 3) ## 3 for men, 10 for women
    
# # # update display output - wins and points - keep updating
for i in range(0, len(disp_results)):
    write_gdoc_cell(int(disp_results.at[i, "Wins"]), output_ws[0], "ScoreDisp", 5+i, 4) ## 4 for men, 11 for women
    write_gdoc_cell(int(disp_results.at[i, "Points"]), output_ws[0], "ScoreDisp", 5+i, 5) ## 5 for men, 12 for women

In [44]:
disp_results

,Team Name,Court Number,Wins,Points,Pool Ranking
0,Boston Hurricanes Black,CT01,5,35,0
1,DC Yee Fung Toy Thunder A,CT01,0,-35,0
2,NJ Ronin,CT01,0,0,0
3,Philly CIA A,CT01,0,-80,0
4,Toronto Connex A,CT01,5,80,0
5,Chicago United,CT02,0,-60,0
6,DC CYC A,CT02,5,10,0
7,Montreal Freemason,CT02,5,60,0
8,Toronto Flying Tigers,CT02,0,0,0
9,Toronto LEGACY,CT02,0,-10,0


### Sort results by court number, wins, points

### Write team names and records to Gold, Silver, Bronze brackets

# Need to adjust
- Gold: top 2 teams from each pool goes into playoffs (total 8)
- Silver: top 2 teams from each pool goes into playoffs (total 6)
- Bronze: top 3 teams from each pool goed into playofffs (total 6)

In [140]:
# sort by court number, wins, points for seeding
results = results.sort_values(by=["Court Number", "Wins", "Points"], ascending=[1,0,0]).reset_index(drop=True)

# assign pool rankings

for court in results["Court Number"].unique():
    # retrieve pool
    pool = results[results["Court Number"] == court]
    # sort by wins and points
    pool = pool.sort_values(by=["Wins", "Points"], ascending=[0,0]).reset_index(drop=True)
    
    # assign rankings
    for i in range(0, len(pool)):
        results.loc[results["Team Name"] == pool.loc[i, "Team Name"], "Pool Ranking"] = i+1

# do not use chained assignments - [] then []
# results["Pool Ranking"][results["Team Name"] == team["Team Name"]] = 4
# use this instead
# results.loc[results["Team Name"] == team["Team Name"], "Pool Ranking"] = 6

mens_gold_courts = ["CT01", "CT02", "CT03", "CT04"]
womens_gold_courts = ["CT15", "CT16", "CT17", "CT18"]
mens_silver_courts = ["CT12", "CT13", "CT14"]
womens_silver_courts = ["CT09", "CT10", "CT11"]
womens_bronze_courts = ["CT05", "CT06"]

mens_gold = results[results["Court Number"].isin(mens_gold_courts)].reset_index(drop=True)

mens_silver = results[results["Court Number"].isin(mens_silver_courts)].reset_index(drop=True)


womens_gold = results[results["Court Number"].isin(womens_gold_courts)].reset_index(drop=True)


womens_silver = results[results["Court Number"].isin(womens_silver_courts)].reset_index(drop=True)

womens_bronze = results[results["Court Number"].isin(womens_bronze_courts)].reset_index(drop=True)

## write team names and records into brackets for gold, silver, women's bronze

ws = "NY Mini 2019 - Day 2 Playoffs"
sheets = ["Men's GOLD", "Men's SILVER"] ## Change for womens
pools = [mens_gold, mens_silver] ## Change for womens

## sheets = ["Women's GOLD", "Women's SILVER", "Women's BRONZE"]
## pool = [womens_gold, womens_silver, womens_bronze]

for sheet in sheets:
    for pool in pools:
        for i in range(0, len(pool)):
            write_gdoc_cell(pool.at[i, "Team Name"], ws, sheet, i+4, 4)
            write_gdoc_cell(int(pool.at[i, "Pool Ranking"]), ws,sheet, i+4, 3)
            write_gdoc_cell(pool.at[i, "Court Number"], ws, sheet, i+4, 2)
            write_gdoc_cell(int(pool.at[i, "Wins"]), ws, sheet, i+4, 5)
            write_gdoc_cell(int(pool.at[i, "Points"]), ws, sheet, i+4, 6)